# Part-of-speech tagging

## Reading data, vocabulary and pretrained embeddings
These parts are similar to the previous examples. Things to note though:
* Our data has already been tokenized and divided into sentences
* We _cannot_ skip tokens
* We are using a specific OOV (out-of-vocabulary) embedding for all words which are not present in our vocab
* We now have one label for each token, not for each document.

In [2]:
# Load our training data
import json
import random
import numpy
with open("data/pos_train.json") as f:
    data=json.load(f)
print(data[0])

# We need to gather the texts, into a list
texts=[one_example["text"] for one_example in data]
labels=[one_example["tags"] for one_example in data] # This is now a list of lists just like the texts variable
print(texts[:2])
print(labels[:2])

# Lets do the same thing for the validation data
# We use a separate validation set, since generally using sentences from the same documents as train/validation results in overly optimistic scores
with open("data/pos_devel.json") as f:
    validation_data=json.load(f)
validation_texts=[one_example["text"] for one_example in data]
validation_labels=[one_example["tags"] for one_example in data]

# Use gensim to read the embedding model

from gensim.models import KeyedVectors

vector_model=KeyedVectors.load_word2vec_format("data/wiki-news-300d-1M.vec", binary=False, limit=50000)

# sort based on the index to make sure they are in the correct order
words=[k for k,v in sorted(vector_model.vocab.items(), key=lambda x:x[1].index)]
print("Words from embedding model:",len(words))
print("First 50 words:",words[:50])

# Normalize the vectors

print("Before normalization:",vector_model.get_vector("in")[:10])
vector_model.init_sims(replace=True)
print("After normalization:",vector_model.get_vector("in")[:10])

# Build vocabulary mappings

vocabulary={"<SPECIAL>": 0, "<OOV>": 1} # zero has a special meaning in sequence models, prevent using it for a normal word
for word in words:
    vocabulary.setdefault(word, len(vocabulary))

print("Words in vocabulary:",len(vocabulary))
inversed_vocabulary={value:key for key, value in vocabulary.items()} # inverse the dictionary

# Label mappings
label_set = set([label for sentence_labels in labels for label in sentence_labels])
label_map = {label: index for index, label in enumerate(label_set)}
                
# Embedding matrix

def load_pretrained_embeddings(vocab, embedding_model):
    """ vocab: vocabulary from our data vectorizer, embedding_model: model loaded with gensim """
    pretrained_embeddings=numpy.random.uniform(low=-0.05, high=0.05, size=(len(vocab)-1,embedding_model.vectors.shape[1]))
    pretrained_embeddings = numpy.vstack((numpy.zeros(shape=(1,embedding_model.vectors.shape[1])), pretrained_embeddings))
    found=0
    for word,idx in vocab.items():
        if word in embedding_model.vocab:
            pretrained_embeddings[idx]=embedding_model.get_vector(word)
            found+=1
            
    print("Found pretrained vectors for {found} words.".format(found=found))
    return pretrained_embeddings

pretrained=load_pretrained_embeddings(vocabulary, vector_model)


{'tags': ['PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'ADJ', 'NOUN', 'VERB', 'PROPN', 'PROPN', 'PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'PROPN', 'PUNCT', 'ADP', 'DET', 'ADJ', 'NOUN', 'PUNCT'], 'text': ['Al', '-', 'Zaman', ':', 'American', 'forces', 'killed', 'Shaikh', 'Abdullah', 'al', '-', 'Ani', ',', 'the', 'preacher', 'at', 'the', 'mosque', 'in', 'the', 'town', 'of', 'Qaim', ',', 'near', 'the', 'Syrian', 'border', '.']}
[['Al', '-', 'Zaman', ':', 'American', 'forces', 'killed', 'Shaikh', 'Abdullah', 'al', '-', 'Ani', ',', 'the', 'preacher', 'at', 'the', 'mosque', 'in', 'the', 'town', 'of', 'Qaim', ',', 'near', 'the', 'Syrian', 'border', '.'], ['[', 'This', 'killing', 'of', 'a', 'respected', 'cleric', 'will', 'be', 'causing', 'us', 'trouble', 'for', 'years', 'to', 'come', '.', ']']]
[['PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'ADJ', 'NOUN', 'VERB', 'PROPN', 'PROPN', 'PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN',

## Vectorizing data
If we want to consider the task as sequence labeling, we should feed the input data as word sequences and outputs as label sequences.

In [3]:
import numpy

def vectorizer(vocab, texts, label_map, labels):
    vectorized_data = [] # turn text into numbers based on our vocabulary mapping
    vectorized_labels = [] # same thing for the labels
    sentence_lengths = [] # Number of tokens in each sentence
    
    for i, one_example in enumerate(texts):
        vectorized_example = []
        vectorized_example_labels = []
        for word in one_example:
            vectorized_example.append(vocab.get(word, 1)) # 1 is our index for out-of-vocabulary tokens
        
        for label in labels[i]:
            vectorized_example_labels.append(label_map[label])

        vectorized_data.append(vectorized_example)
        vectorized_labels.append(vectorized_example_labels)
        
        sentence_lengths.append(len(one_example))
        
    vectorized_data = numpy.array(vectorized_data) # turn python list into numpy matrix
    vectorized_labels = numpy.array(vectorized_labels)
    
    return vectorized_data, vectorized_labels, sentence_lengths

vectorized_data, vectorized_labels, lengths=vectorizer(vocabulary, texts, label_map, labels)
validation_vectorized_data, validation_vectorized_labels, validation_lengths=vectorizer(vocabulary, validation_texts, label_map, validation_labels)

## Padding
We add padding to the label sequences as well.

In [4]:
import tensorflow as tf
### Only needed for me, not to block the whole GPU, you don't need this stuff
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))
### ---end of weird stuff

from keras.preprocessing.sequence import pad_sequences
print("Old shape:", vectorized_data.shape)
vectorized_data_padded=pad_sequences(vectorized_data, padding='post')
print("New shape:", vectorized_data_padded.shape)
print("First example:", vectorized_data_padded[0])
# Even with the sparse output format, the shape has to be similar to the one-hot encoding
vectorized_labels_padded=numpy.expand_dims(pad_sequences(vectorized_labels, padding='post'), -1)
print("Padded labels shape:", vectorized_labels_padded.shape)
print(label_map)
print("First example labels:", vectorized_labels_padded[0])

weights = numpy.copy(vectorized_data_padded)
weights[weights > 0] = 1
print("First weight vector:", weights[0])

# Same stuff for the validation data
validation_vectorized_data_padded=pad_sequences(validation_vectorized_data, padding='post')
validation_vectorized_labels_padded=numpy.expand_dims(pad_sequences(validation_vectorized_labels, padding='post'), -1)
validation_weights = numpy.copy(validation_vectorized_data_padded)
validation_weights[weights > 0] = 1

/home/kahaka/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Old shape: (12543,)
New shape: (12543, 159)
First example: [ 3424    37     1    11   285  1084   974 34462 10554  4733    37 43264
     2     3 16500    29     3  8683     8     3   754     6     1     2
   504     3  4761  1757     4     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     

## Evaluating POS tags
Keras does not use sample weighting in metrics (only for losses) (correct me if I'm wrong), so we have to create our own evaluation if we want to ignore padding in models which do not support masking (e.g. convolution).
Thus, to have evaluation that is identical for all models, we have to create our own script, which will ignore padded parts of the sequences.


In [42]:
import keras
from sklearn.metrics import accuracy_score

def accuracy(predictions, gold, lengths):
    pred_tags = numpy.concatenate([labels[:lengths[i]] for i, labels in enumerate(predictions)]).ravel()
    
    gold_tags = numpy.concatenate([labels[:lengths[i], 0] for i, labels in enumerate(gold)]).ravel()
    
    print('Accuracy:', accuracy_score(gold_tags, pred_tags))

class EvaluateTags(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        pred = numpy.argmax(self.model.predict(validation_vectorized_data_padded), axis=-1)
        accuracy(pred, validation_vectorized_labels_padded, validation_lengths) # FIXME: Using global variables here, not good!

## Independent classification
Time-distributed means that the same dense layer is applied to each time step. This means that we are now simply using a normal feedforward network to classify each word/token separately.

Why didn't we one-hot encode our labels? :S

It's because the sparse loss is doing it for us implicitly! Neat, right!

In [43]:
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Activation, Conv1D, TimeDistributed, LSTM, Bidirectional
from keras.optimizers import SGD, Adam

example_count, sequence_len = vectorized_data_padded.shape
class_count = len(label_set)

vector_size= pretrained.shape[1]

In [ ]:
inp=Input(shape=(sequence_len,))
embeddings=Embedding(len(vocabulary), vector_size, mask_zero=True, trainable=False, weights=[pretrained])(inp)
hidden = TimeDistributed(Dense(100, activation="softmax"))(embeddings)
outp = TimeDistributed(Dense(class_count, activation="softmax"))(hidden)
model=Model(inputs=[inp], outputs=[outp])

optimizer=Adam(lr=0.001) # define the learning rate
model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy", sample_weight_mode='temporal')

print(model.summary())

# train
hist=model.fit(vectorized_data_padded,vectorized_labels_padded, sample_weight=weights, batch_size=100,verbose=1,epochs=100, callbacks=[EvaluateTags()])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 159)               0         
_________________________________________________________________
embedding_7 (Embedding)      (None, 159, 300)          15000600  
_________________________________________________________________
time_distributed_13 (TimeDis (None, 159, 100)          30100     
_________________________________________________________________
time_distributed_14 (TimeDis (None, 159, 17)           1717      
Total params: 15,032,417
Trainable params: 31,817
Non-trainable params: 15,000,600
_________________________________________________________________
None
Epoch 1/100
12543/12543 [==============================] - 8s 647us/step - loss: 27.0499
Accuracy: 0.30482828055736116
Epoch 2/100
12543/12543 [==============================] - 8s 641us/step - loss: 25.7625
Accuracy: 0.3310297301656346
Epoch 3/100
125

# Adding context with convolution

In [11]:
inp=Input(shape=(sequence_len,))
embeddings=Embedding(len(vocabulary), vector_size, mask_zero=False, trainable=False, weights=[pretrained])(inp)
cnn = Conv1D(100,3, activation='relu', padding='same')(embeddings)
outp=TimeDistributed(Dense(class_count, activation="softmax"))(cnn)
model=Model(inputs=[inp], outputs=[outp])

optimizer=Adam(lr=0.001) # define the learning rate
model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy", sample_weight_mode='temporal')

print(model.summary())

# train
hist=model.fit(vectorized_data_padded,vectorized_labels_padded, sample_weight=weights, batch_size=100,verbose=1,epochs=100, callbacks=[EvaluateEntities()])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 113)               0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 113, 300)          15000600  
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 113, 100)          90100     
_________________________________________________________________
time_distributed_7 (TimeDist (None, 113, 4)            404       
Total params: 15,091,104
Trainable params: 90,504
Non-trainable params: 15,000,600
_________________________________________________________________
None
Epoch 1/100
13800/14041 [============================>.] - ETA: 0s - loss: 0.4250
Precision/Recall/F-score: 0.6337763075343638 / 0.5548786577449316 / 0.591709044436753
14041/14041 [==============================] - 8s 590us/step - loss: 0.4213
Epoch 2/

Epoch 31/100
14000/14041 [============================>.] - ETA: 0s - loss: 0.0622
Precision/Recall/F-score: 0.8357740080228119 / 0.8635274143613303 / 0.8494240734828205
14041/14041 [==============================] - 4s 304us/step - loss: 0.0622
Epoch 32/100
14000/14041 [============================>.] - ETA: 0s - loss: 0.0610
Precision/Recall/F-score: 0.8401530192242507 / 0.8663737141715769 / 0.8530619268874302
14041/14041 [==============================] - 5s 362us/step - loss: 0.0610
Epoch 33/100
13400/14041 [===========================>..] - ETA: 0s - loss: 0.0605
Precision/Recall/F-score: 0.829796325309785 / 0.8727654049735344 / 0.8507386405120592
14041/14041 [==============================] - 4s 305us/step - loss: 0.0604
Epoch 34/100
13400/14041 [===========================>..] - ETA: 0s - loss: 0.0597
Precision/Recall/F-score: 0.8773286285091588 / 0.8442524717866773 / 0.8604728096292338
14041/14041 [==============================] - 4s 288us/step - loss: 0.0596
Epoch 35/100
1360

Epoch 98/100
13800/14041 [============================>.] - ETA: 0s - loss: 0.0395
Precision/Recall/F-score: 0.8865501074428599 / 0.9064715869369819 / 0.896400177768999
14041/14041 [==============================] - 5s 337us/step - loss: 0.0396
Epoch 99/100
14000/14041 [============================>.] - ETA: 0s - loss: 0.0396
Precision/Recall/F-score: 0.8894181193223668 / 0.9044741835613702 / 0.8968829689782378
14041/14041 [==============================] - 4s 317us/step - loss: 0.0396
Epoch 100/100
13500/14041 [===========================>..] - ETA: 0s - loss: 0.0395
Precision/Recall/F-score: 0.9203599317370843 / 0.8886946968940378 / 0.9042501841831162
14041/14041 [==============================] - 4s 308us/step - loss: 0.0393


## LSTMs

In [12]:
inp=Input(shape=(sequence_len,))
embeddings=Embedding(len(vocabulary), vector_size, mask_zero=True, trainable=False, weights=[pretrained])(inp)
rnn = LSTM(100, activation='tanh', return_sequences=True)(embeddings)
outp=TimeDistributed(Dense(class_count, activation="softmax"))(rnn)
model=Model(inputs=[inp], outputs=[outp])

optimizer=Adam(lr=0.001) # define the learning rate
model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy", sample_weight_mode='temporal')

print(model.summary())

# train
hist=model.fit(vectorized_data_padded,vectorized_labels_padded, sample_weight=weights, batch_size=100,verbose=1,epochs=100, callbacks=[EvaluateEntities()])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 113)               0         
_________________________________________________________________
embedding_7 (Embedding)      (None, 113, 300)          15000600  
_________________________________________________________________
lstm_3 (LSTM)                (None, 113, 100)          160400    
_________________________________________________________________
time_distributed_8 (TimeDist (None, 113, 4)            404       
Total params: 15,161,404
Trainable params: 160,804
Non-trainable params: 15,000,600
_________________________________________________________________
None
Epoch 1/100
14000/14041 [============================>.] - ETA: 0s - loss: 4.3386
Precision/Recall/F-score: 0.3169291338582677 / 0.03215819434734845 / 0.058391513283162576
14041/14041 [==============================] - 155s 11ms/step - loss: 4.3347
Ep

Epoch 31/100
14000/14041 [============================>.] - ETA: 0s - loss: 0.5584
Precision/Recall/F-score: 0.8812849921182802 / 0.8095975232198143 / 0.8439216094526715
14041/14041 [==============================] - 137s 10ms/step - loss: 0.5584
Epoch 32/100
14000/14041 [============================>.] - ETA: 0s - loss: 0.5413
Precision/Recall/F-score: 0.8880721683929168 / 0.7963647258563867 / 0.8397219882055602
14041/14041 [==============================] - 140s 10ms/step - loss: 0.5416
Epoch 33/100
14000/14041 [============================>.] - ETA: 0s - loss: 0.5332
Precision/Recall/F-score: 0.9040313549832026 / 0.8062518725656647 / 0.8523465132238821
14041/14041 [==============================] - 149s 11ms/step - loss: 0.5330
Epoch 34/100
14000/14041 [============================>.] - ETA: 0s - loss: 0.5196
Precision/Recall/F-score: 0.8840437158469945 / 0.807849795266154 / 0.8442310702917079
14041/14041 [==============================] - 140s 10ms/step - loss: 0.5201
Epoch 35/100


14000/14041 [============================>.] - ETA: 0s - loss: 0.3415
Precision/Recall/F-score: 0.9560114023810855 / 0.854089683411565 / 0.9021810797267718
14041/14041 [==============================] - 148s 11ms/step - loss: 0.3419
Epoch 98/100
14000/14041 [============================>.] - ETA: 0s - loss: 0.4030
Precision/Recall/F-score: 0.9419622810956444 / 0.8380105862378907 / 0.8869510068178215
14041/14041 [==============================] - 148s 11ms/step - loss: 0.4025
Epoch 99/100
14000/14041 [============================>.] - ETA: 0s - loss: 0.3613
Precision/Recall/F-score: 0.9579342407438526 / 0.8539898132427843 / 0.9029805433089575
14041/14041 [==============================] - 147s 10ms/step - loss: 0.3616
Epoch 100/100
14000/14041 [============================>.] - ETA: 0s - loss: 0.3438
Precision/Recall/F-score: 0.9592362394310991 / 0.8554379306901029 / 0.9043684835686946
14041/14041 [==============================] - 146s 10ms/step - loss: 0.3439


## Bidirectional LSTM

In [14]:
inp=Input(shape=(sequence_len,))
embeddings=Embedding(len(vocabulary), vector_size, mask_zero=True, trainable=False, weights=[pretrained])(inp)
rnn = Bidirectional(LSTM(100, activation='tanh', return_sequences=True))(embeddings)
outp=TimeDistributed(Dense(class_count, activation="softmax"))(rnn)
model=Model(inputs=[inp], outputs=[outp])

optimizer=Adam(lr=0.001) # define the learning rate
model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy", sample_weight_mode='temporal')

print(model.summary())

# train
hist=model.fit(vectorized_data_padded,vectorized_labels_padded, sample_weight=weights, batch_size=100,verbose=1,epochs=10, callbacks=[EvaluateEntities()])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 113)               0         
_________________________________________________________________
embedding_9 (Embedding)      (None, 113, 300)          15000600  
_________________________________________________________________
bidirectional_2 (Bidirection (None, 113, 200)          320800    
_________________________________________________________________
time_distributed_10 (TimeDis (None, 113, 4)            804       
Total params: 15,322,204
Trainable params: 321,604
Non-trainable params: 15,000,600
_________________________________________________________________
None
Epoch 1/10
14000/14041 [============================>.] - ETA: 0s - loss: 3.9394
Precision/Recall/F-score: 0.553005284015852 / 0.16723259762308998 / 0.2568054597040104
14041/14041 [==============================] - 307s 22ms/step - loss: 3.9332
Epoch 